# PCA on multiple expressions meshes

## Process
- READ OBJs systematically
    - Make them functions to be called later
- RUN PCA (sklearn PCA)
    - Make them functions to be called later
    - __(+30Oct) Take into account only vertices which represebts the front face region which effectively used for facial expressions__
- Save in .pkl

## requirements/test-environment
- Manjaro Linux
- python 3.12.0
- Anaconda 22.9.0
    - packages-list
        - see requirement.txt

# Read OBJs
- place downloaded tracked meshes of multiple expressions /dataset/multiface/tracked_mesh/

In [ ]:
import sys
import os
# set path to dataset
path_to_dataset = os.path.join(os.getcwd(), '../dataset/multiface/tracked_mesh/')
save_path = "../dataset/multiface/tracked_mesh"

ID = 6795937

In [ ]:
# create a list of expressions
list_exps_name = []
map_exps2id = {}
counter = 0
for i, name in enumerate(os.listdir(path_to_dataset)):
    f = os.path.join(path_to_dataset, name)
    if os.path.isdir(f) and name.startswith('E0'):
        counter = counter + 1
        list_exps_name.append(name)


list_exps_name.sort()

for i, exp_name in enumerate(list_exps_name):
    print(f'{i}, {exp_name}')
    map_exps2id.update({exp_name: i})




Read OBJ in ./dataset/multiface

In [ ]:
# list of expressions
print(list_exps_name)
print(map_exps2id)

File handler for .obj in ./utils/FolderHandler.py

In [ ]:
from utils.Dataset_handler import Filehandler

In [ ]:
file_handler = Filehandler(path_to_dataset=path_to_dataset)
print(file_handler.get_path_to_dataset())
file_handler.iter_dir()
print("Expressions: Number of tracked mesh")
for key in file_handler.dict_objs.keys():
    print(f'{list_exps_name[key]}: {len(file_handler.dict_objs[key])}')
    print(file_handler.dict_objs[key]) 
# print(file_handler.dict_objs)

OBJ class

In [ ]:
from utils.OBJ_helper import OBJ

Test .obj loading

In [ ]:
test_path = "../dataset/multiface/tracked_mesh/E003_Neutral_Eyes_Closed/000783.obj"
test_obj = OBJ(test_path, swapyz=False)
print(f'Number of vertices: {len(test_obj.vertices)}')
print(test_obj.vertices)

load all .obj in the certain directory

In [ ]:
dict_expOBJs = {}
dict_expVerts = {}

# selecgt number of samples for a expression 
num_samples_perExp = 20

for expID, key in enumerate(file_handler.dict_objs.keys()):
    list_OBJs = []
    list_Verts = []

    # since there are many sequences for a expression, we assume that second half of tracked mesh in a sequence captured the specific expressions
    # We only use second half of sequence for a expression.
    # This is also for resonable memory usage as well. if you run over all, you will consume more than 30GB memory to store all of objects

    # half_id = int(len(file_handler.dict_objs[key])/2)
    # end_id = int(len(file_handler.dict_objs[key]))

    for i, obj in enumerate(file_handler.dict_objs[key][0:num_samples_perExp]):
        path_to_obj = os.path.join(file_handler.list_expPathFiles[expID], obj)
        # print(path_to_obj)
        obj = OBJ(path_to_obj, swapyz=False)
        list_OBJs.append(obj)
        list_Verts.append(obj.vertices)
    dict_expOBJs.update({expID: list_OBJs})
    dict_expVerts.update({expID: list_Verts})

check the number of objs

In [ ]:
# for key in dict_expVerts.keys():
#     vertices = dict_expVerts[key]
#     print(len(vertices))

Test conversion from list to numpy array

In [ ]:
exp0_vertices = dict_expVerts[0]
print(f"Expression: {list_exps_name[0]}")
print(f"The number of meshes: {len(exp0_vertices)}")
print(f"The number of vertices for each mesh: {len(exp0_vertices[0])}")

In [ ]:
import numpy as np
import pandas as pd
num_vertices = 7306
len_col = num_vertices*3
print(f"The number of vertices for each meesh: {num_vertices}")
print(f"The number of columns of matrix X: {len_col}")

Test concatenation and check it

In [ ]:
_np_array = np.array(exp0_vertices)
print(_np_array.shape)
print(_np_array[0][0][0:3])

In [ ]:
_array = np.array(exp0_vertices).reshape((_np_array.shape[0], len_col))
print(_array.shape)
print(_array[0][0:3])

Concatenation of vertex positions for all tracked meshes of all expressions

In [ ]:
_list_xs = []
num_sum_samples = 0
for key in dict_expVerts.keys():
    vertices = dict_expVerts[key]
    _num_samples = len(vertices)
    print(_num_samples)
    num_sum_samples = num_sum_samples + _num_samples
    _array = np.array(vertices).reshape((_num_samples, len_col))
    _list_xs.append(_array)

# print(f"Len of _list_xs: {len(_list_xs)}")
# print(f"The number of samples: {num_sum_samples}")

In [ ]:
neutralmesh_verts = _list_xs[0]
X = _list_xs[0]
for x in _list_xs[1:]:
    X = np.concatenate((X, x), axis = 0)
    # print(X)
print(X.shape)


Obtain the mean vertex coordinates over the sequence in "E001_Neutral_Eyes_Opens"
- We assume that the average vertex coordinate over the sequence in "E001_Neutral_Eyes_Open" defines the ID's neutral face mesh

In [ ]:
ave_neutralmesh_vertices = np.mean(neutralmesh_verts, axis = 0)
print(ave_neutralmesh_vertices.shape)

For PCA, we need to centrailzed data at the neutral face which is defined like above not at the average expression
- We need to subtract the matrix from the average vertex coordinates of the sequence of neutral face ("E001_Neutral_Eyes_Opened")

In [ ]:
cent_X = X-ave_neutralmesh_vertices[None, :]

Look at the contents in matrix X

In [ ]:
# df = pd.DataFrame(X)
# print(df)

# Face mask
- Masking the vertices to take into account only vertices composing the front face as you see below in blue

- To do it, we only perform over these blue vertices.
![face_mask_region](../images/Facemask_side01.png)

- Approach
    - In the data matrix, we replace vertices coordinate with 0 over the vertices composing other region 

In [ ]:
from utils.Blendshape import FaceMask
from utils.pickel_io import dump_pckl, load_from_memory

# set the name of pickel file to be loaded
pickel_fname = "FaceMask_30102023_09_40.pkl"

facemask = load_from_memory(path_to_memory = save_path, pickle_fname = pickel_fname)


In [ ]:
masked_cent_X = cent_X * facemask.bit_mask[None, :]

In [ ]:
# check if the data matrix is masked
# we chose the ID:3567 which is located on top of nose 
# as the center of facemesh which is obtained by the nearest neighboring search at the vertex

print(sum(masked_cent_X[:,3567*3]) != 0)
print(sum(masked_cent_X[:,3567*3+1]) != 0)
print(sum(masked_cent_X[:,3567*3+2]) != 0)

## PCA on tracked meshes (min(#samples, #vertices) selected expressions)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
# The dimensions of space where can be represented by linear combination of principal components
D = masked_cent_X.shape[0]

In [ ]:
if D == -1:
    pca = PCA() #D=min(feature dimensions, the number of samples)
else:
    pca = PCA(D) #D

pca.fit(masked_cent_X)
Gamma = pca.components_ #right-hand side matrix of eigenvalue decomposition
Variance = pca.explained_variance_
Stds = np.sqrt(Variance)
Sigma = np.diag(pca.explained_variance_) #diagonal matrix which has eigenvectors in each diagonal entry
MEAN =  ave_neutralmesh_vertices # neutral face mesh (vertex coodinates of neutral face)

In [ ]:
print(f"Shape of Gamma (right-handside): {Gamma.shape}")
# print(Gamma)
print(f"Shape of Sigma (diagonal matrix): {Sigma.shape}")
# print(Sigma)
print(f"Shape of MEAN (average expressions): {MEAN.shape}")

## check the reconstruction error (E_RMS)
- Root mean squared error (divided by 1000 as convenience)

In [ ]:
masked_cent_X.shape

In [ ]:
# Sparsity check
# close to 1: Sparse, close to 0: Dense
sparsity_level = np.mean(Gamma==0)
print(sparsity_level)

# Visualization of sparsity of the principle components

In [ ]:
x_data = np.arange(0,Gamma.shape[1], 1)
y_data = Gamma.T
plt.plot(x_data, y_data[:,:3])
plt.xlabel("Vertices")
plt.ylabel("Displacement")
plt.title("Components in matrix V (3 components from first row)")
plt.show()

# visualize the flipped cumulative sum over obtained principal components
- As the corresponding value (y-axis) becomes lager, the component has larger global effect

$$std_j = 1-  \frac{std_i}{\sum_j^{100}std_{j}}$$

In [ ]:
sum_std = Stds.sum()
print(sum_std)
df_std = pd.DataFrame((Stds/sum_std))
cumsum_std=df_std.cumsum()
negative_cumsum_std = 1-cumsum_std
negative_cumsum_std[:100].plot()
plt.ylabel("std_deviations")
plt.xlabel("principal components")
# plt.xticks(np.arange(0, len(Stds)-1, 5))
plt.show()

# Create a new expressions using PCs and Stds
- Sample a vector of coefficients from a normal distribution not to deviate from mean
D: how many principal components will be employed to represents a normal distribution

$$a_i \sim \mathcal{N}(0, \sigma_i^2)$$
$$i = \{0, D_{pcs}\}$$

In [ ]:
# select D
D_Pcs = 5

In [ ]:
coefficients = []
for i, std in enumerate(Stds[:D_Pcs]):
    mu,sigma = 0, std # mean and standard deviation
    _noise = np.random.normal(mu, sigma, 1)
    coefficients.append(_noise)

In [ ]:
print(coefficients)

add linear combination of principal components, in which are weighted by coefficients to mean vertex position

In [ ]:
newFace_vertices = MEAN
for coeff in coefficients:
    _item = coeff*Gamma[i]
    newFace_vertices = newFace_vertices + _item

In [ ]:
print(newFace_vertices)
print(newFace_vertices.shape)
print(newFace_vertices.shape[0]/3)

write a .obj based on obtained vertex positions for visualization
- Since the tracked mesh are topologically equivalent, we can easily obtain .obj only by rewriting line for vertex position (starting with 'v')

In [ ]:
# path configuration
sample_path = "../dataset/multiface/tracked_mesh/sample.obj"

ourput pointclouds (.obj) based on obtained vertex coordinates
>>check /dataset/multiface/tracked_mesh/result_point_clouds.obj

In [ ]:
# Sample obj file should be read first
obj = OBJ(sample_path, swapyz=False)
obj.write_PointClouds(save_path, newFace_vertices, mesh_name = "result_point_clouds")

ourput mesh (.obj) based on obtained vertex coordinates
>>check /dataset/multiface/tracked_mesh/result_mesh.obj

In [ ]:
# Sample obj file should be read first
OBJ.write_OBJfile(reference_obj_file=sample_path, save_path=save_path, vertices=newFace_vertices, name_Exp="all-5Pcs")

# Interpretation of each blendshape
- Output each blendshape
- Compare with average face

In [ ]:
from utils.Blendshape import Blendshape

obtain each blenshape (.obj)

In [ ]:
blendshapes = []

if not os.path.exists(os.path.join(save_path, "First4Pcs_test")):
    os.mkdir(os.path.join(save_path, "First4Pcs_test"))


OBJ.write_OBJfile(reference_obj_file= sample_path, save_path = save_path, vertices=MEAN, name_Exp="averageExp")

for i in range(1, 5):
    _blendshape = Blendshape(Verts_averageExp=MEAN, PCs=Gamma, Stds = Stds, D = i, save_path=os.path.join(save_path, "First4Pcs_test"), only_specific_pc=True, name_newExp=str(i))
    _blendshape.sample_coefficients()
    _blendshape.get_newExp()
    fname = _blendshape.generate_newExp()
    blendshapes.append(_blendshape)

dump PCs, Stds, MEAN in .pickel for visualization

In [ ]:
from utils.Blendshape import datastruct_blendshape

print(type(MEAN))
print(type(Gamma))
print(type(Stds))
print(save_path)

In [ ]:
from datetime import datetime
# get current date and year
now = datetime.now()

date = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y")
print(date)
time = now.strftime("%H_%M")
print("time:", time)

In [ ]:
our_blendshape = datastruct_blendshape(ID = ID, List_exps = list_exps_name, MEAN = MEAN, PCs = Gamma, Stds = Stds)
# print(our_blendshape)
pickel_fname = 'blendshape_'+date+'_'+time+'.pkl'
# dump datastruct_blendshape
dump_pckl(data=our_blendshape, save_root=save_path, pickel_fname=pickel_fname)

In [ ]:
print(our_blendshape)

load blendshape from memory (pickel)

In [ ]:
load_from_memory(path_to_memory=save_path, pickle_fname=pickel_fname)